In [1]:
%load_ext rpy2.ipython

In [21]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)
library(dplyr)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"

GTEx_DATA_DIR <- IN_DIR
GTEx_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GTEx_DATA_DIR,GTEx_DATA_FIL,sep=""))

In [ ]:
%%R

variances <- apply(assay(final_result), 1, var)

zero_variance_rows <- which(variances == 0)

if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data_10.RData"))
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    final_result <- new_final_result
    
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R
library(tibble)

keep_samples <- colData(final_result)$dose %in% c(0.00, 30.00)

final_result <- final_result[, keep_samples]

temp_df <- as_tibble(colData(final_result)) %>%
  filter(!grepl("SRP131784", study))

tcdd_data <- temp_df
final_result <- final_result[, colData(final_result)$external_id %in% temp_df$external_id]

print(dim(final_result))
print(dim(tcdd_data))


[1] 43252   136
[1] 136 198


In [ ]:
%%R
library(dplyr)
library(SummarizedExperiment)

sampling_percentage <- 0.10

tcdd_data_dose0 <- filter(tcdd_data, dose == 0)
tcdd_data_dose30 <- filter(tcdd_data, dose == 30)

min_count <- min(nrow(tcdd_data_dose0), nrow(tcdd_data_dose30)) * sampling_percentage
min_count <- floor(min_count) 

sampled_tcdd_dose0 <- sample_n(tcdd_data_dose0, size = min_count)
sampled_tcdd_dose30 <- sample_n(tcdd_data_dose30, size = min_count)

tcdd_data <- bind_rows(sampled_tcdd_dose0, sampled_tcdd_dose30)

final_result <- final_result[, colData(final_result)$external_id %in% tcdd_data$external_id]
matched_indices <- match(tcdd_data$external_id, colData(final_result)$external_id)
final_result <- final_result[, matched_indices]

print(dim(tcdd_data))
print(dim(final_result))


[1]  12 198
[1] 43252    12


In [ ]:
%%R
dose_counts <- table(tcdd_data$dose)

print(dose_counts)

dose_counts_df <- as.data.frame(dose_counts)

colnames(dose_counts_df) <- c("Dose", "Count")

write.csv(dose_counts_df, paste0(OUT_DIR, "dose_counts030_time_course_10.csv"), row.names=FALSE)


 0 30 
 6  6 


In [26]:
%%R
combined_gender <- ifelse(tcdd_data$Sex != "", tcdd_data$Sex, tcdd_data$gender)
print(combined_gender)

 [1] "male"   "male"   "male"   "female" "female" "male"   "female" "female"
 [9] "male"   "male"   "male"   "male"  


In [27]:
%%R
write.table(tcdd_data$project_id,file=paste(OUT_DIR,"tcdd_project_id030_time_course_10.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(combined_gender,file=paste(OUT_DIR,"tcdd_gender030_time_course_10.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_id030_time_course_10.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose030_time_course_10.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
saveRDS(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose_detail_vec030_time_course_10.Rds",sep=""))
saveRDS(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_detail_vec030_time_course_10.Rds",sep=""))


In [28]:
%%R
tcdd.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()
print(tcdd.df)

                           V1      V2      V3       V4       V5      V6
ENSMUSG00000079800.2        0       0       0        0       50       0
ENSMUSG00000095092.1        0       0       0        0        0       0
ENSMUSG00000079192.2        0       0       0        0        0       0
ENSMUSG00000079794.2        0       0       0        0        0       0
ENSMUSG00000094799.1        0       0       0        0        0       0
ENSMUSG00000095250.1        0       0       0        0        0       0
ENSMUSG00000095787.1        0       0       0        0        0       0
ENSMUSG00000096100.1        0       0       0        0        0       0
ENSMUSG00000094054.1        0       0       0        0        0       0
ENSMUSG00000095672.1        0       0       0        0        0       0
ENSMUSG00000079190.3        0       0       0      100        0       0
ENSMUSG00000094514.1        0       0      50       98        0       0
ENSMUSG00000094915.1        0       0       0        0        0 

ENSMUSG00000103884.1        0       0       0        0        0       0
ENSMUSG00000084353.1     3948    1763    4360     2278     1699    2391
ENSMUSG00000103933.1    33604   23201   37416    36425    16021   23644
ENSMUSG00000076135.1        0       8      50        0        0       0
ENSMUSG00000086195.1       10       6      64       35        0      20
ENSMUSG00000104504.1        7      18       6        6        6      18
ENSMUSG00000066693.2     3214    1635    2963     3116     2767    3353
ENSMUSG00000102316.1        0       0       0        0        0     100
ENSMUSG00000103819.1        0       0       0        0        0       0
ENSMUSG00000102871.1        0       0       0        0        0       0
ENSMUSG00000025909.16      50     100     341        0      183      50
ENSMUSG00000101571.2      350     150     200      312      307     150
ENSMUSG00000104428.1        0       0       0        0        0       0
ENSMUSG00000102272.1     1240    1830    2175     1684     2616 

ENSMUSG00000091020.3     1497    1446    2044     3139     1242    1499
ENSMUSG00000101640.1        0       0       0        0        0       0
ENSMUSG00000042686.5      450     450     649      783      796     998
ENSMUSG00000025777.8        0       0       0        0        0       0
ENSMUSG00000099899.1        0       0       0        0        0       0
ENSMUSG00000100398.1      347     200     346      300      150     648
ENSMUSG00000101589.1     2026    1140    2769     2443     2964    2140
ENSMUSG00000102048.1      100     150     300      350       45       0
ENSMUSG00000067780.3        0      50       0      100      250       0
ENSMUSG00000100053.1        0       0       0        0        0       0
ENSMUSG00000085125.7        0       0       0        0        0       0
ENSMUSG00000025776.13       0       0       0        0        0      46
ENSMUSG00000099895.1        0       0       0        0        0       0
ENSMUSG00000100554.1      649     566    1089      889      786 

ENSMUSG00000080391.1        0       0       0        0        0       0
ENSMUSG00000099492.1    15393    8033   13490    15304     8312   13370
ENSMUSG00000101726.1        0      50      50        0       49       0
ENSMUSG00000100261.1     1982    1987    2808     2934     2616    2986
ENSMUSG00000101180.1        0       0       0        0        0       0
ENSMUSG00000103783.1        0       0       0      149       50       0
ENSMUSG00000104315.1        0       0       0        0      100       0
ENSMUSG00000048874.15   24280   19858   25059    20323    29153   23462
ENSMUSG00000026064.16   28954   38770   48932    36854   118487   33901
ENSMUSG00000117310.2    28699   37776   46408    36332   116956   32708
ENSMUSG00000117091.1        0       0       0        0        0       0
ENSMUSG00000086726.3        0       0       0        0        0       0
ENSMUSG00000100237.1        0       0       0        0       84       0
ENSMUSG00000100910.1        0       0       0        0        0 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

     1981
ENSMUSG00000020267.6    109527    64407   134907   138953   142989   111117
ENSMUSG00000018339.11     3900     3578    56373    37779   102266    10931
ENSMUSG00000082180.1       526      383      680      584      586      528
ENSMUSG00000020400.17     5906     4368    17661    11693    18045    11368
ENSMUSG00000018340.13   132116    59540    57971    39931    67850    42991
ENSMUSG00000049588.13      349       49      598      186      433      250
ENSMUSG00000000594.7     89807    56702    48892    28126    54787    46559
ENSMUSG00000044751.5    159652    91365   127541    95638   121451   134750
ENSMUSG00000080738.1     56178     4078     1187      684      430     2320
ENSMUSG00000085531.1         0        0        0      100      200        0
ENSMUSG00000049491.15       50        0        0        0       50       50
ENSMUSG00000020264.5       150      200     1339      650     1956      400
ENSMUSG00000084980.1         0        0        0       50        0        0
EN

ENSMUSG00000009900.7         0        0        0        0        0        0
ENSMUSG00000000126.11      650      250      450      250      547        0
ENSMUSG00000093572.1         0        0        0        0        0        0
ENSMUSG00000049291.6         0        0        0        0        0        0
ENSMUSG00000086910.1         0        0        0        0        0        0
ENSMUSG00000009894.15    43063    22614    24737    22992    28143    19525
ENSMUSG00000036819.14     8258     5315     6427     5761     6438     5800
ENSMUSG00000054519.8      9656     3515     4318     4582     4637     3119
ENSMUSG00000084145.3      5374     3545    13207    10300    12914     6984
ENSMUSG00000083234.1         0        0        0        0        0        0
ENSMUSG00000072893.12     5289     3126     4116     4303     6617     3311
ENSMUSG00000020472.14    11918     6355    11548     7761    15494     7929
ENSMUSG00000032691.14      708      743     3163     2984     3936     1597
ENSMUSG00000

ENSMUSG00000018500.2        50        0      350      399      499      400
ENSMUSG00000014243.5      6578     2987     4647     4546     5091     4273
ENSMUSG00000081340.1         0        0       50       21      200       26
ENSMUSG00000042298.18    18702    11734    28071    26641    26575    17726
ENSMUSG00000081944.1       916      464      626      418      296      381
ENSMUSG00000018501.17    67355    42084    79331    72071    79586    53020
ENSMUSG00000086310.1        35        0        0        0        0        0
ENSMUSG00000014245.8      5746     3099     9839     9388    12182     7346
ENSMUSG00000086636.1         0        0        0        0        0        0
ENSMUSG00000018509.8      4929     2604     4804     7067     4958     5467
ENSMUSG00000087120.1       164       80      173       87      148      210
ENSMUSG00000019505.7    461599   250702   586188   515202   550234   406139
ENSMUSG00000085748.1         0        0        0        0        0        0
ENSMUSG00000

ENSMUSG00000052921.13     7571     5548     9940     7739    10155     6855
ENSMUSG00000018740.14     2197     1446      899     1098      782      350
ENSMUSG00000032892.14      600      532      521      461      366      151
ENSMUSG00000092727.1        18       36       20        6        0        0
ENSMUSG00000084522.1         0       19        0        0        0        0
ENSMUSG00000020899.16    11403     6287     7463     7339     8999     6132
ENSMUSG00000020898.18     5748     4111     8351     6028     8441     5720
ENSMUSG00000020897.12      273      293     1570     1941     1547     1992
ENSMUSG00000072809.2      1964     1310     1872     1752     3431     2445
ENSMUSG00000045176.3      4631     2747     4985     6370     8733     5580
ENSMUSG00000020895.14     1137      635      783      755      993      592
ENSMUSG00000064899.1       132      139        0        0        0        0
ENSMUSG00000084069.1         0        0        0        0        0        0
ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
%%R
print(dim(tcdd.df))
setdiff(rownames(tcdd.df), rownames(final_result))

In [30]:
%%R
colnames(tcdd.df) <- colData(final_result)$external_id
print(tcdd.df)


ENSMUSG00000102626.1           0         25          0          0          0
ENSMUSG00000085361.2           0          0          0         50          0
ENSMUSG00000102157.1           0          0          0          0          0
ENSMUSG00000105936.1           0          0          0          0         50
ENSMUSG00000102451.1           0        115         43         49         19
ENSMUSG00000085481.1           0          0          0          0          0
ENSMUSG00000102243.1           0         50        100          0         50
ENSMUSG00000056268.15       6089       6969      11798       7783       8772
ENSMUSG00000063681.14          0          0          0          0          0
ENSMUSG00000033964.12       6970       6129      10537       7539      13885
ENSMUSG00000033952.14        350        149         50        200        150
ENSMUSG00000026368.6       88723     105493      93922     101034     134194
ENSMUSG00000057037.8       71743      80028      74676      63798     10414

ENSMUSG00000103928.1        1345       1531       1590       1298       2455
ENSMUSG00000053286.11      11333       9598       9362      11166      10912
ENSMUSG00000103238.1           0          0          0         99          0
ENSMUSG00000026484.13       6282       5569       8681       7604      14101
ENSMUSG00000100981.1           0          0          0          0          0
ENSMUSG00000100551.1        3886       1355       4209       7083       4483
ENSMUSG00000026483.13        799        439       3684       2039       1397
ENSMUSG00000100594.1        1448       1748       1568       2649       1044
ENSMUSG00000043019.12      14042      12988      15219      13279      19467
ENSMUSG00000084557.1           0          0          0          0          0
ENSMUSG00000100685.1           0          0          0          9          0
ENSMUSG00000032666.16        797        498       1993       1946       1787
ENSMUSG00000102090.1           0          0          0          0          0

ENSMUSG00000101905.1           0          0          0          0          0
ENSMUSG00000103325.1           0          0          0          0          0
ENSMUSG00000101768.1           0          0          0          0          0
ENSMUSG00000026592.13          0          0        100          0          0
ENSMUSG00000109913.1           0          0          0          0          0
ENSMUSG00000102326.1           0          0          0          0        100
ENSMUSG00000102391.1           0          0          0         50          0
ENSMUSG00000087632.1           0          0          0          0         50
ENSMUSG00000103808.1           0          0          0          0          0
ENSMUSG00000070565.11       4531       3555       5121       5156       7694
ENSMUSG00000049881.13       2464       1293       1903       1721       1036
ENSMUSG00000033488.11       7889       7713       8626      11341       7343
ENSMUSG00000087093.2           0          0          0          0          0

ENSMUSG00000078190.7         100         98        500        150        249
ENSMUSG00000040265.16        186        194        391        835        390
ENSMUSG00000066752.4          50          0          0        177        100
ENSMUSG00000026694.19       5262       6373       5211       8550       5258
ENSMUSG00000026696.15       3859       5612       5782       5800       7043
ENSMUSG00000091743.2           0          0          0          0          0
ENSMUSG00000026697.10          0          0          0          0          0
ENSMUSG00000091060.2           0          0          0          0          0
ENSMUSG00000098259.1           0         40         35          0         69
ENSMUSG00000040225.15      20161      16077      24599      27481      24339
ENSMUSG00000026692.12       4542       5143       5423       9488      11670
ENSMUSG00000040181.14     152151     104950     103060     185455     235332
ENSMUSG00000040170.13       8727       8878       6424      22430      62031

ENSMUSG00000102548.1           0          0          0          0          0
ENSMUSG00000097823.2           0         50          0         47        100
ENSMUSG00000060568.14         50          0          0         50          0
ENSMUSG00000102870.1           0          0          0          0          0
ENSMUSG00000103333.1           0          0          0          0          0
ENSMUSG00000103139.1           0          0          0          0          0
ENSMUSG00000026558.13       5635       4212       4228       5788       3695
ENSMUSG00000102909.1           0          0          0          0          0
ENSMUSG00000052428.10      37581      35217      43996      47194      44898
ENSMUSG00000088246.1           9         64          0       8101       1219
ENSMUSG00000095752.1           0          0          0          0          0
ENSMUSG00000026687.14     122214     120146      99546     185363     133130
ENSMUSG00000026688.5       10615       7776      11363      12696      11114

...

ENSMUSG00000086799.1         150          0
ENSMUSG00000020530.14      64934      35227
ENSMUSG00000045140.8        6317       5430
ENSMUSG00000092644.1           0          0
ENSMUSG00000020527.14      12192      10606
ENSMUSG00000020526.12       6121       3565
ENSMUSG00000000805.18          0          0
ENSMUSG00000082917.1           0          0
ENSMUSG00000083876.1 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



          0
ENSMUSG00000020576.10      22469      14426
ENSMUSG00000113666.1           0          0
ENSMUSG00000113283.1           0          0
ENSMUSG00000097973.1         249        239
ENSMUSG00000113863.1           0          0
ENSMUSG00000113397.1          50          0
ENSMUSG00000113855.1         100          0
ENSMUSG00000113522.1           0          0
ENSMUSG00000113167.1           0          0
ENSMUSG00000020607.7        1598       1199
ENSMUSG00000113336.1           0          0
ENSMUSG00000113961.1           0          0
ENSMUSG00000050974.4         541        393
ENSMUSG00000075760.1           0          0
ENSMUSG00000113663.1           0          0
ENSMUSG00000114066.1        1340       1117
ENSMUSG00000113644.1           2         49
ENSMUSG00000114124.1         185        322
ENSMUSG00000113285.1         348         46
ENSMUSG00000113734.1           0          0
ENSMUSG00000113069.1           0          0
ENSMUSG00000113339.1           0          0
ENSMUSG00000098158.2

ENSMUSG00000066553.5        4077       2136
ENSMUSG00000020653.12      10985       7776
ENSMUSG00000062563.15       1994       1296
ENSMUSG00000020649.11       1836       6308
ENSMUSG00000113062.1           0         50
ENSMUSG00000114076.1           0          0
ENSMUSG00000113018.1           0          0
ENSMUSG00000114084.1           0          0
ENSMUSG00000113621.1           0          0
ENSMUSG00000020646.17        245         40
ENSMUSG00000113774.1         280         12
ENSMUSG00000036333.11     133927      71177
ENSMUSG00000114148.1         100          0
ENSMUSG00000020644.9       71647      47850
ENSMUSG00000112937.1          50        200
ENSMUSG00000114033.1           0         50
ENSMUSG00000113587.1           0        150
ENSMUSG00000113745.1           0          0
ENSMUSG00000114105.1           0         11
ENSMUSG00000113669.1        1192        899
ENSMUSG00000114111.1           0          0
ENSMUSG00000113611.1         924        896
ENSMUSG00000113550.1           0

ENSMUSG00000055917.15       7595       4205
ENSMUSG00000035954.10      28559      20493
ENSMUSG00000113435.1           0          0
ENSMUSG00000113510.1        2328       3245
ENSMUSG00000113337.1           0          0
ENSMUSG00000114166.1          51          0
ENSMUSG00000114130.1          49         49
ENSMUSG00000113994.1           0          0
ENSMUSG00000113086.1           0          0
ENSMUSG00000077613.1           0          0
ENSMUSG00000113454.1           0          0
ENSMUSG00000113516.1           0          0
ENSMUSG00000036295.5         400        500
ENSMUSG00000113766.1           0          0
ENSMUSG00000113438.1           0          0
ENSMUSG00000113105.1           0          0
ENSMUSG00000056899.10      10095       8913
ENSMUSG00000113370.1           0          0
ENSMUSG00000113979.1           0          0
ENSMUSG00000113989.1           0          0
ENSMUSG00000112407.1           0          0
ENSMUSG00000065004.1           0         50
ENSMUSG00000112844.1           0

In [31]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(tcdd.df))
# print(ensembl_wo_ids)

In [32]:
%%R
deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))
print(length(deleted_ensembl_wo_ids))

[1] 12169


In [33]:
%%R
rownames(tcdd.df) <- ensembl_wo_ids
# print(rownames(tcdd.df))

In [34]:
%%R
rownames(deleted_data) <- deleted_ensembl_wo_ids
print(rownames(deleted_data))

    [1] "ENSMUSG00000096776" "ENSMUSG00000096244" "ENSMUSG00000095450"
    [4] "ENSMUSG00000089163" "ENSMUSG00000099278" "ENSMUSG00000094121"
    [7] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000094741"
   [10] "ENSMUSG00000074720" "ENSMUSG00000094337" "ENSMUSG00000095570"
   [13] "ENSMUSG00000098647" "ENSMUSG00000094431" "ENSMUSG00000095247"
   [16] "ENSMUSG00000095585" "ENSMUSG00000096873" "ENSMUSG00000095755"
   [19] "ENSMUSG00000095207" "ENSMUSG00000093828" "ENSMUSG00000095623"
   [22] "ENSMUSG00000095666" "ENSMUSG00000096680" "ENSMUSG00000096236"
   [25] "ENSMUSG00000096756" "ENSMUSG00000094722" "ENSMUSG00000095728"
   [28] "ENSMUSG00000094836" "ENSMUSG00000094383" "ENSMUSG00000094474"
   [31] "ENSMUSG00000096550" "ENSMUSG00000094172" "ENSMUSG00000091585"
   [34] "ENSMUSG00000095763" "ENSMUSG00000095523" "ENSMUSG00000095475"
   [37] "ENSMUSG00000095019" "ENSMUSG00000102693" "ENSMUSG00000064842"
   [40] "ENSMUSG00000102851" "ENSMUSG00000103377" "ENSMUSG00000102348"
   [43

  [346] "ENSMUSG00000104104" "ENSMUSG00000087883" "ENSMUSG00000099892"
  [349] "ENSMUSG00000100977" "ENSMUSG00000077039" "ENSMUSG00000100684"
  [352] "ENSMUSG00000099770" "ENSMUSG00000088510" "ENSMUSG00000101890"
  [355] "ENSMUSG00000087908" "ENSMUSG00000088594" "ENSMUSG00000100955"
  [358] "ENSMUSG00000099469" "ENSMUSG00000101297" "ENSMUSG00000100059"
  [361] "ENSMUSG00000088161" "ENSMUSG00000100935" "ENSMUSG00000103991"
  [364] "ENSMUSG00000084704" "ENSMUSG00000103060" "ENSMUSG00000089866"
  [367] "ENSMUSG00000089206" "ENSMUSG00000084228" "ENSMUSG00000044594"
  [370] "ENSMUSG00000058017" "ENSMUSG00000080729" "ENSMUSG00000073602"
  [373] "ENSMUSG00000073601" "ENSMUSG00000087210" "ENSMUSG00000089278"
  [376] "ENSMUSG00000077237" "ENSMUSG00000099484" "ENSMUSG00000104478"
  [379] "ENSMUSG00000101231" "ENSMUSG00000100884" "ENSMUSG00000089037"
  [382] "ENSMUSG00000084741" "ENSMUSG00000100290" "ENSMUSG00000100758"
  [385] "ENSMUSG00000102728" "ENSMUSG00000088691" "ENSMUSG00000099884"
  [388

  [691] "ENSMUSG00000077749" "ENSMUSG00000111753" "ENSMUSG00000088423"
  [694] "ENSMUSG00000044499" "ENSMUSG00000103299" "ENSMUSG00000111549"
  [697] "ENSMUSG00000096080" "ENSMUSG00000077576" "ENSMUSG00000111234"
  [700] "ENSMUSG00000088925" "ENSMUSG00000111048" "ENSMUSG00000111208"
  [703] "ENSMUSG00000110782" "ENSMUSG00000094521" "ENSMUSG00000094311"
  [706] "ENSMUSG00000111183" "ENSMUSG00000019834" "ENSMUSG00000111098"
  [709] "ENSMUSG00000110889" "ENSMUSG00000046922" "ENSMUSG00000111576"
  [712] "ENSMUSG00000094663" "ENSMUSG00000111315" "ENSMUSG00000093298"
  [715] "ENSMUSG00000087891" "ENSMUSG00000099893" "ENSMUSG00000098642"
  [718] "ENSMUSG00000076282" "ENSMUSG00000110891" "ENSMUSG00000077082"
  [721] "ENSMUSG00000098418" "ENSMUSG00000077591" "ENSMUSG00000098993"
  [724] "ENSMUSG00000098409" "ENSMUSG00000111182" "ENSMUSG00000094681"
  [727] "ENSMUSG00000065718" "ENSMUSG00000111499" "ENSMUSG00000111419"
  [730] "ENSMUSG00000110843" "ENSMUSG00000111719" "ENSMUSG00000112062"
  [733

 [1036] "ENSMUSG00000107400" "ENSMUSG00000094734" "ENSMUSG00000108114"
 [1039] "ENSMUSG00000058513" "ENSMUSG00000088147" "ENSMUSG00000096858"
 [1042] "ENSMUSG00000094702" "ENSMUSG00000071065" "ENSMUSG00000050478"
 [1045] "ENSMUSG00000069421" "ENSMUSG00000063173" "ENSMUSG00000049052"
 [1048] "ENSMUSG00000052818" "ENSMUSG00000061961" "ENSMUSG00000107399"
 [1051] "ENSMUSG00000058071" "ENSMUSG00000094295" "ENSMUSG00000095608"
 [1054] "ENSMUSG00000107423" "ENSMUSG00000048745" "ENSMUSG00000044897"
 [1057] "ENSMUSG00000108253" "ENSMUSG00000095804" "ENSMUSG00000112688"
 [1060] "ENSMUSG00000059862" "ENSMUSG00000045559" "ENSMUSG00000088352"
 [1063] "ENSMUSG00000085559" "ENSMUSG00000085512" "ENSMUSG00000064632"
 [1066] "ENSMUSG00000077304" "ENSMUSG00000088688" "ENSMUSG00000080833"
 [1069] "ENSMUSG00000086808" "ENSMUSG00000082897" "ENSMUSG00000098853"
 [1072] "ENSMUSG00000084318" "ENSMUSG00000089247" "ENSMUSG00000104137"
 [1075] "ENSMUSG00000088861" "ENSMUSG00000064766" "ENSMUSG00000087990"
 [1078

 [1381] "ENSMUSG00000086617" "ENSMUSG00000084200" "ENSMUSG00000064788"
 [1384] "ENSMUSG00000000125" "ENSMUSG00000080694" "ENSMUSG00000089479"
 [1387] "ENSMUSG00000086089" "ENSMUSG00000087184" "ENSMUSG00000088279"
 [1390] "ENSMUSG00000098989" "ENSMUSG00000064691" "ENSMUSG00000087139"
 [1393] "ENSMUSG00000085324" "ENSMUSG00000020624" "ENSMUSG00000086760"
 [1396] "ENSMUSG00000086135" "ENSMUSG00000078600" "ENSMUSG00000075431"
 [1399] "ENSMUSG00000089460" "ENSMUSG00000084130" "ENSMUSG00000064588"
 [1402] "ENSMUSG00000081318" "ENSMUSG00000078586" "ENSMUSG00000086588"
 [1405] "ENSMUSG00000085336" "ENSMUSG00000110653" "ENSMUSG00000082562"
 [1408] "ENSMUSG00000085059" "ENSMUSG00000084887" "ENSMUSG00000080616"
 [1411] "ENSMUSG00000083155" "ENSMUSG00000099690" "ENSMUSG00000085970"
 [1414] "ENSMUSG00000112240" "ENSMUSG00000092168" "ENSMUSG00000112723"
 [1417] "ENSMUSG00000112559" "ENSMUSG00000088209" "ENSMUSG00000102096"
 [1420] "ENSMUSG00000099360" "ENSMUSG00000077259" "ENSMUSG00000112787"
 [1423

...

 [4729] "ENSMUSG00000070943" "ENSMUSG00000055838" "ENSMUSG00000075379"
 [4732] "ENSMUSG00000081737" "ENSMUSG00000081174" "ENSMUSG00000083361"
 [4735] "ENSMUSG00000068948" "ENSMUSG00000075377" "ENSMUSG00000049094"
 [4738] "ENSMUSG00000064468" "ENSMUSG00000087062" "ENSMUSG00000083722"
 [4741] "ENSMUSG00000087164" "ENSMUSG00000083401" "ENSMUSG00000081839"
 [4744] "ENSMUSG00000084285" "ENSMUSG00000082015" "ENSMUSG00000108203"
 [4747] "ENSMUSG00000084171" "ENSMUSG00000081577" "ENSMUSG00000088512"
 [4750] "ENSMUSG00000098953" "ENSMUSG00000098660" "ENSMUSG00000049252"
 [4753] "ENSMUSG00000105844" "ENSMUSG00000077351" "ENSMUSG00000086830"
 [4756] "ENSMUSG00000088868" "ENSMUSG00000089144" "ENSMUSG00000077769"
 [4759] "ENSMUSG00000085357" "ENSMUSG00000088825" "ENSMUSG00000100496"
 [4762] "ENSMUSG00000086007" "ENSMUSG00000089614" "ENSMUSG00000083472"
 [4765] "ENSMUSG00000056115" "ENSMUSG00000081484" "ENSMUSG00000089609"
 [4768] "ENSMUSG00000088928" "ENSMUSG00000088628" "ENSMUSG00000064596"
 [4771

 [5074] "ENSMUSG00000075067" "ENSMUSG00000082574" "ENSMUSG00000091703"
 [5077] "ENSMUSG00000075065" "ENSMUSG00000075063" "ENSMUSG00000075061"
 [5080] "ENSMUSG00000099223" "ENSMUSG00000098406" "ENSMUSG00000086980"
 [5083] "ENSMUSG00000087559" "ENSMUSG00000089073" "ENSMUSG00000106336"
 [5086] "ENSMUSG00000065511" "ENSMUSG00000106196" "ENSMUSG00000077678"
 [5089] "ENSMUSG00000087069" "ENSMUSG00000077281" "ENSMUSG00000084906"
 [5092] "ENSMUSG00000083846" "ENSMUSG00000087537" "ENSMUSG00000104258"
 [5095] "ENSMUSG00000091288" "ENSMUSG00000090449" "ENSMUSG00000091061"
 [5098] "ENSMUSG00000084460" "ENSMUSG00000088633" "ENSMUSG00000088624"
 [5101] "ENSMUSG00000105133" "ENSMUSG00000101186" "ENSMUSG00000082435"
 [5104] "ENSMUSG00000087465" "ENSMUSG00000084385" "ENSMUSG00000102686"
 [5107] "ENSMUSG00000083643" "ENSMUSG00000089606" "ENSMUSG00000088700"
 [5110] "ENSMUSG00000077236" "ENSMUSG00000082934" "ENSMUSG00000086764"
 [5113] "ENSMUSG00000086029" "ENSMUSG00000087226" "ENSMUSG00000081966"
 [5116

 [5419] "ENSMUSG00000102233" "ENSMUSG00000103275" "ENSMUSG00000103607"
 [5422] "ENSMUSG00000102203" "ENSMUSG00000087945" "ENSMUSG00000102397"
 [5425] "ENSMUSG00000074664" "ENSMUSG00000080368" "ENSMUSG00000105806"
 [5428] "ENSMUSG00000109716" "ENSMUSG00000092990" "ENSMUSG00000076120"
 [5431] "ENSMUSG00000098315" "ENSMUSG00000103095" "ENSMUSG00000102884"
 [5434] "ENSMUSG00000102256" "ENSMUSG00000103499" "ENSMUSG00000084665"
 [5437] "ENSMUSG00000102498" "ENSMUSG00000103896" "ENSMUSG00000098622"
 [5440] "ENSMUSG00000088260" "ENSMUSG00000104203" "ENSMUSG00000105475"
 [5443] "ENSMUSG00000106584" "ENSMUSG00000077559" "ENSMUSG00000105577"
 [5446] "ENSMUSG00000098795" "ENSMUSG00000105222" "ENSMUSG00000104797"
 [5449] "ENSMUSG00000084530" "ENSMUSG00000104901" "ENSMUSG00000098254"
 [5452] "ENSMUSG00000105284" "ENSMUSG00000105101" "ENSMUSG00000114832"
 [5455] "ENSMUSG00000098862" "ENSMUSG00000106512" "ENSMUSG00000064834"
 [5458] "ENSMUSG00000106180" "ENSMUSG00000105281" "ENSMUSG00000083857"
 [5461

 [5764] "ENSMUSG00000106359" "ENSMUSG00000082720" "ENSMUSG00000106614"
 [5767] "ENSMUSG00000064569" "ENSMUSG00000092718" "ENSMUSG00000083717"
 [5770] "ENSMUSG00000065872" "ENSMUSG00000105121" "ENSMUSG00000077253"
 [5773] "ENSMUSG00000105882" "ENSMUSG00000104711" "ENSMUSG00000089017"
 [5776] "ENSMUSG00000105507" "ENSMUSG00000105073" "ENSMUSG00000105665"
 [5779] "ENSMUSG00000106452" "ENSMUSG00000077410" "ENSMUSG00000105225"
 [5782] "ENSMUSG00000088346" "ENSMUSG00000080676" "ENSMUSG00000077741"
 [5785] "ENSMUSG00000089060" "ENSMUSG00000088840" "ENSMUSG00000106342"
 [5788] "ENSMUSG00000092666" "ENSMUSG00000093054" "ENSMUSG00000104703"
 [5791] "ENSMUSG00000065569" "ENSMUSG00000093000" "ENSMUSG00000087806"
 [5794] "ENSMUSG00000065684" "ENSMUSG00000089027" "ENSMUSG00000105207"
 [5797] "ENSMUSG00000098097" "ENSMUSG00000104749" "ENSMUSG00000105767"
 [5800] "ENSMUSG00000092934" "ENSMUSG00000076949" "ENSMUSG00000080545"
 [5803] "ENSMUSG00000106238" "ENSMUSG00000104531" "ENSMUSG00000064915"
 [5806

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [35]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)

[5695] "ENSMUSG00000035845" "ENSMUSG00000035847" "ENSMUSG00000035864"
[5698] "ENSMUSG00000035878" "ENSMUSG00000035885" "ENSMUSG00000035890"
[5701] "ENSMUSG00000035891" "ENSMUSG00000035898" "ENSMUSG00000035901"
[5704] "ENSMUSG00000035919" "ENSMUSG00000035923" "ENSMUSG00000035929"
[5707] "ENSMUSG00000035930" "ENSMUSG00000035933" "ENSMUSG00000035936"
[5710] "ENSMUSG00000035948" "ENSMUSG00000035949" "ENSMUSG00000035953"
[5713] "ENSMUSG00000035954" "ENSMUSG00000035958" "ENSMUSG00000035960"
[5716] "ENSMUSG00000035992" "ENSMUSG00000036006" "ENSMUSG00000036023"
[5719] "ENSMUSG00000036040" "ENSMUSG00000036053" "ENSMUSG00000036061"
[5722] "ENSMUSG00000036067" "ENSMUSG00000036073" "ENSMUSG00000036083"
[5725] "ENSMUSG00000036091" "ENSMUSG00000036095" "ENSMUSG00000036104"
[5728] "ENSMUSG00000036106" "ENSMUSG00000036111" "ENSMUSG00000036112"
[5731] "ENSMUSG00000036117" "ENSMUSG00000036123" "ENSMUSG00000036138"
[5734] "ENSMUSG00000036155" "ENSMUSG00000036158" "ENSMUSG00000036180"
[5737] "ENSMUSG00000

[6265] "ENSMUSG00000039115" "ENSMUSG00000039116" "ENSMUSG00000039117"
[6268] "ENSMUSG00000039137" "ENSMUSG00000039148" "ENSMUSG00000039153"
[6271] "ENSMUSG00000039159" "ENSMUSG00000039187" "ENSMUSG00000039191"
[6274] "ENSMUSG00000039193" "ENSMUSG00000039194" "ENSMUSG00000039196"
[6277] "ENSMUSG00000039197" "ENSMUSG00000039201" "ENSMUSG00000039206"
[6280] "ENSMUSG00000039209" "ENSMUSG00000039217" "ENSMUSG00000039218"
[6283] "ENSMUSG00000039219" "ENSMUSG00000039221" "ENSMUSG00000039224"
[6286] "ENSMUSG00000039231" "ENSMUSG00000039234" "ENSMUSG00000039238"
[6289] "ENSMUSG00000039239" "ENSMUSG00000039242" "ENSMUSG00000039252"
[6292] "ENSMUSG00000039253" "ENSMUSG00000039254" "ENSMUSG00000039275"
[6295] "ENSMUSG00000039298" "ENSMUSG00000039304" "ENSMUSG00000039308"
[6298] "ENSMUSG00000039318" "ENSMUSG00000039321" "ENSMUSG00000039323"
[6301] "ENSMUSG00000039345" "ENSMUSG00000039347" "ENSMUSG00000039356"
[6304] "ENSMUSG00000039357" "ENSMUSG00000039361" "ENSMUSG00000039367"
[6307] "ENSMUSG00000

[6616] "ENSMUSG00000041354" "ENSMUSG00000041408" "ENSMUSG00000041417"
[6619] "ENSMUSG00000041426" "ENSMUSG00000041429" "ENSMUSG00000041431"
[6622] "ENSMUSG00000041438" "ENSMUSG00000041444" "ENSMUSG00000041453"
[6625] "ENSMUSG00000041477" "ENSMUSG00000041488" "ENSMUSG00000041491"
[6628] "ENSMUSG00000041498" "ENSMUSG00000041506" "ENSMUSG00000041528"
[6631] "ENSMUSG00000041530" "ENSMUSG00000041534" "ENSMUSG00000041538"
[6634] "ENSMUSG00000041548" "ENSMUSG00000041550" "ENSMUSG00000041556"
[6637] "ENSMUSG00000041559" "ENSMUSG00000041577" "ENSMUSG00000041592"
[6640] "ENSMUSG00000041598" "ENSMUSG00000041608" "ENSMUSG00000041616"
[6643] "ENSMUSG00000041624" "ENSMUSG00000041632" "ENSMUSG00000041638"
[6646] "ENSMUSG00000041642" "ENSMUSG00000041644" "ENSMUSG00000041650"
[6649] "ENSMUSG00000041653" "ENSMUSG00000041658" "ENSMUSG00000041660"
[6652] "ENSMUSG00000041670" "ENSMUSG00000041681" "ENSMUSG00000041685"
[6655] "ENSMUSG00000041688" "ENSMUSG00000041695" "ENSMUSG00000041697"
[6658] "ENSMUSG00000

[6967] "ENSMUSG00000045136" "ENSMUSG00000045140" "ENSMUSG00000045210"
[6970] "ENSMUSG00000045211" "ENSMUSG00000045216" "ENSMUSG00000045232"
[6973] "ENSMUSG00000045236" "ENSMUSG00000045246" "ENSMUSG00000045248"
[6976] "ENSMUSG00000045251" "ENSMUSG00000045273" "ENSMUSG00000045281"
[6979] "ENSMUSG00000045282" "ENSMUSG00000045288" "ENSMUSG00000045302"
[6982] "ENSMUSG00000045312" "ENSMUSG00000045316" "ENSMUSG00000045318"
[6985] "ENSMUSG00000045322" "ENSMUSG00000045328" "ENSMUSG00000045382"
[6988] "ENSMUSG00000045394" "ENSMUSG00000045404" "ENSMUSG00000045409"
[6991] "ENSMUSG00000045427" "ENSMUSG00000045438" "ENSMUSG00000045467"
[6994] "ENSMUSG00000045471" "ENSMUSG00000045482" "ENSMUSG00000045502"
[6997] "ENSMUSG00000045503" "ENSMUSG00000045509" "ENSMUSG00000045515"
[7000] "ENSMUSG00000045518" "ENSMUSG00000045534" "ENSMUSG00000045545"
[7003] "ENSMUSG00000045551" "ENSMUSG00000045573" "ENSMUSG00000045594"
[7006] "ENSMUSG00000045613" "ENSMUSG00000045625" "ENSMUSG00000045662"
[7009] "ENSMUSG00000

[7318] "ENSMUSG00000049858" "ENSMUSG00000049882" "ENSMUSG00000049908"
[7321] "ENSMUSG00000049922" "ENSMUSG00000049928" "ENSMUSG00000049929"
[7324] "ENSMUSG00000049932" "ENSMUSG00000049940" "ENSMUSG00000049950"
[7327] "ENSMUSG00000049960" "ENSMUSG00000050014" "ENSMUSG00000050017"
[7330] "ENSMUSG00000050028" "ENSMUSG00000050029" "ENSMUSG00000050069"
[7333] "ENSMUSG00000050088" "ENSMUSG00000050097" "ENSMUSG00000050103"
[7336] "ENSMUSG00000050132" "ENSMUSG00000050138" "ENSMUSG00000050144"
[7339] "ENSMUSG00000050147" "ENSMUSG00000050148" "ENSMUSG00000050150"
[7342] "ENSMUSG00000050158" "ENSMUSG00000050164" "ENSMUSG00000050188"
[7345] "ENSMUSG00000050192" "ENSMUSG00000050211" "ENSMUSG00000050213"
[7348] "ENSMUSG00000050229" "ENSMUSG00000050232" "ENSMUSG00000050234"
[7351] "ENSMUSG00000050244" "ENSMUSG00000050248" "ENSMUSG00000050271"
[7354] "ENSMUSG00000050272" "ENSMUSG00000050288" "ENSMUSG00000050295"
[7357] "ENSMUSG00000050296" "ENSMUSG00000050304" "ENSMUSG00000050310"
[7360] "ENSMUSG00000

[7669] "ENSMUSG00000054662" "ENSMUSG00000054693" "ENSMUSG00000054702"
[7672] "ENSMUSG00000054717" "ENSMUSG00000054720" "ENSMUSG00000054733"
[7675] "ENSMUSG00000054757" "ENSMUSG00000054759" "ENSMUSG00000054763"
[7678] "ENSMUSG00000054764" "ENSMUSG00000054766" "ENSMUSG00000054808"
[7681] "ENSMUSG00000054823" "ENSMUSG00000054855" "ENSMUSG00000054889"
[7684] "ENSMUSG00000054892" "ENSMUSG00000054893" "ENSMUSG00000054894"
[7687] "ENSMUSG00000054901" "ENSMUSG00000054920" "ENSMUSG00000054931"
[7690] "ENSMUSG00000054932" "ENSMUSG00000054934" "ENSMUSG00000054942"
[7693] "ENSMUSG00000054958" "ENSMUSG00000054967" "ENSMUSG00000054978"
[7696] "ENSMUSG00000055022" "ENSMUSG00000055024" "ENSMUSG00000055026"
[7699] "ENSMUSG00000055027" "ENSMUSG00000055041" "ENSMUSG00000055065"
[7702] "ENSMUSG00000055067" "ENSMUSG00000055069" "ENSMUSG00000055078"
[7705] "ENSMUSG00000055088" "ENSMUSG00000055116" "ENSMUSG00000055150"
[7708] "ENSMUSG00000055170" "ENSMUSG00000055172" "ENSMUSG00000055202"
[7711] "ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids030_10.Rds",sep=""))

In [37]:
%%R
tcdd.df <- tcdd.df[reactome_ensembl_ids,]
print(tcdd.df)

                   SRR7817684 SRR7817648 SRR4317607 SRR3593524 SRR1636591
ENSMUSG00000000001      45215      34038      32779      39272      51776
ENSMUSG00000000028        396        326        565        480        735
ENSMUSG00000000049     842516     787453     987356    1085031     962984
ENSMUSG00000000058        842        694       2050       1241       2326
ENSMUSG00000000085       2985       3783       5525       5387       3845
ENSMUSG00000000088      72628      79085      34683      58937      41416
ENSMUSG00000000120       1532        797       1496       1842        450
ENSMUSG00000000126        200        200        248        346         50
ENSMUSG00000000127       1764       2174       2229       2819       3147
ENSMUSG00000000142       3818       2480       4059      13762       3785
ENSMUSG00000000149      66074      75215      52516      41734      31987
ENSMUSG00000000154      44822      43741      55444      44412      30260
ENSMUSG00000000168      24024      269

ENSMUSG00000001105      16822      11398      17343      15812      17849
ENSMUSG00000001119       2056       1130       4095       4503       2344
ENSMUSG00000001123     150867     140772     220767     291100     151121
ENSMUSG00000001127      30025      34617      41694      42630      30228
ENSMUSG00000001128       3428       3251      19022      12554       5185
ENSMUSG00000001131          0          0        100         50          0
ENSMUSG00000001143       7878       3959       6465      10943       4482
ENSMUSG00000001155     251639     266164     263628     243672     177096
ENSMUSG00000001158       4764       3712       6408       9030       8631
ENSMUSG00000001173       2451       2528       3698       4824       5104
ENSMUSG00000001175     109063      87815      56091      91837      74794
ENSMUSG00000001211     105667      92305      99165     126397      59860
ENSMUSG00000001225          0          0          0          0         50
ENSMUSG00000001228        250         

ENSMUSG00000002249       5351       5234       3846       6548       3178
ENSMUSG00000002250       9570       3525       3300       9694       2248
ENSMUSG00000002257        150         67        249        385          0
ENSMUSG00000002279      15336      15263      20892      18145      12908
ENSMUSG00000002289      22944      25772      68127      44110      37661
ENSMUSG00000002297       1348        520       1789        448       1376
ENSMUSG00000002307       6556       5149       7102      10790       5064
ENSMUSG00000002308       2342       1942       1567       3069       1731
ENSMUSG00000002324        350        800       1925       1123        585
ENSMUSG00000002325      14588      11143      15486      25160      16029
ENSMUSG00000002326       5960       5842       6474       9360       7195
ENSMUSG00000002346      60392      47013      45885      72060      31734
ENSMUSG00000002365      19343      21719      18860      23024      18488
ENSMUSG00000002379      20241      190

ENSMUSG00000003379        100        100        591        557        150
ENSMUSG00000003402      57123      47146      65502      79187      38875
ENSMUSG00000003411         50        140        182        150          0
ENSMUSG00000003418          0          0        150          0        100
ENSMUSG00000003421       6911      10862       9428       9869       4881
ENSMUSG00000003429      27775      14302      40206      29733      31079
ENSMUSG00000003435      38160      25352      29933      43046      19336
ENSMUSG00000003436          0          0          0          0          0
ENSMUSG00000003437      12789      14126      12208      19580      15385
ENSMUSG00000003444       2625       1464       2294       2154       1196
ENSMUSG00000003452         50          0         50        146         50
ENSMUSG00000003458      20137      20973      25451      29003      19899
ENSMUSG00000003464      44486      49827      67349      88767      58062
ENSMUSG00000003476        100         

ENSMUSG00000004654          0         50          0         43          0
ENSMUSG00000004655      27423      14368      53126      54059      29481
ENSMUSG00000004665       1097        800       1634       1641       1113
ENSMUSG00000004667      11136      11699      10834      12291       6344
ENSMUSG00000004668          0          0        148          0          0
ENSMUSG00000004677      14682      15590      12736      14740       8777
ENSMUSG00000004709        200          0        200        200         50
ENSMUSG00000004730       1568       1043      15645       5949       3634
ENSMUSG00000004768       3192       3933       3937       3227       5073
ENSMUSG00000004771      24212      24509      19236      26311      27315
ENSMUSG00000004788       8090       9015       6912       8727       5217
ENSMUSG00000004789     107720      86252      98472      96676      82710
ENSMUSG00000004791          0          0          0          0          0
ENSMUSG00000004815      20469      169

...

ENSMUSG00000027398       2375       1075
ENSMUSG00000027399       5734       4020
ENSMUSG00000027400          0          0
ENSMUSG00000027404      53742      35353
ENSMUSG00000027405      23578      17623
ENSMUSG00000027406     191864     158034
ENSMUSG00000027412      21035      20352
ENSMUSG00000027419          0        100
ENSMUSG00000027423      83859      55260
ENSMUSG00000027425      24671      19134
ENSMUSG00000027427      10243       6293
ENSMUSG00000027433      47629      28734
ENSMUSG00000027434          0          0
ENSMUSG00000027435      69792      23955
ENSMUSG00000027438       2859       1568
ENSMUSG00000027447      91323      66342
ENSMUSG00000027452       1241        993
ENSMUSG00000027454       1332       1497
ENSMUSG00000027455      25175      21197
ENSMUSG00000027460          7        150
ENSMUSG00000027463       1848        600
ENSMUSG00000027465      77505      38427
ENSMUSG00000027466      78242      60526
ENSMUSG00000027469       2328       6534
ENSMUSG000000274

ENSMUSG00000028031        550        150
ENSMUSG00000028032       9581       5175
ENSMUSG00000028033         50         50
ENSMUSG00000028036         50        150
ENSMUSG00000028039          0          0
ENSMUSG00000028040        150        150
ENSMUSG00000028041      16521       8176
ENSMUSG00000028044       3880       3977
ENSMUSG00000028047       1279        644
ENSMUSG00000028048      38569      20525
ENSMUSG00000028051       1469       1994
ENSMUSG00000028053      38634      21001
ENSMUSG00000028059      20945      10014
ENSMUSG00000028062      22192      21505
ENSMUSG00000028063      66533      54478
ENSMUSG00000028064      18510      26948
ENSMUSG00000028066       5821       8680
ENSMUSG00000028068       1256       3414
ENSMUSG00000028070      32078      21492
ENSMUSG00000028071        835        648
ENSMUSG00000028072          0          0
ENSMUSG00000028076     107951     150031
ENSMUSG00000028081      65190      60935
ENSMUSG00000028082     108992      76641
ENSMUSG000000280

ENSMUSG00000028630      19578      14426
ENSMUSG00000028631        860       1044
ENSMUSG00000028633      38768      24804
ENSMUSG00000028634       3393        900
ENSMUSG00000028635          0          0
ENSMUSG00000028636      12404      21146
ENSMUSG00000028639      92411      79406
ENSMUSG00000028640         50          0
ENSMUSG00000028645      15722       7393
ENSMUSG00000028646      74049      44857
ENSMUSG00000028648       2190       1450
ENSMUSG00000028655       6518      14970
ENSMUSG00000028656     110960      57649
ENSMUSG00000028657      38403      29736
ENSMUSG00000028661        150          0
ENSMUSG00000028664       1235        198
ENSMUSG00000028668      33582      17903
ENSMUSG00000028670      38440      29845
ENSMUSG00000028671      14050      10367
ENSMUSG00000028672     100782      91225
ENSMUSG00000028673      73633      63917
ENSMUSG00000028675      37154      26323
ENSMUSG00000028676      30880      29672
ENSMUSG00000028677      28581      14646
ENSMUSG000000286

ENSMUSG00000029186      39355      34873
ENSMUSG00000029188       1234       1247
ENSMUSG00000029191      23309      15189
ENSMUSG00000029192      22650      12197
ENSMUSG00000029193

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
%%R
saveRDS(tcdd.df,file=paste(OUT_DIR,"tcdd_df030_time_course_10.Rds",sep=""))

In [39]:
%%R
saveRDS(deleted_data, file=paste(OUT_DIR, "format_name_deleted_data030_time_course_10.Rds", sep=""))